In [1]:
import pandas as pd
import json
from openai import OpenAI
import os
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob
import jsonlines

In [2]:
import base64
import requests

# OpenAI API Key
api_key = "OPENAI_API_KEY"

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}
NUM_SECONDS_TO_SLEEP = 10
def get_eval(content: str, base64_image: str, max_tokens: int):
  response = None
  while True:
    try:
      payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
          {
            "role": "user",
            "content": [
              {
                "type": "text",
                "text": content
              },
              {
                "type": "image_url",
                "image_url": {
                  "url": f"data:image/jpeg;base64,{base64_image}"
                }
              }
            ]
          }
        ],
        "max_tokens": max_tokens
      }
      response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
      break
    except openai.error.RateLimitError:
      pass
    except Exception as e:
      print(e)
    time.sleep(NUM_SECONDS_TO_SLEEP)
  # return response.json()['choices'][0]['message']['content']
  return response.json()['choices'][0]['message']['content']



## 이미지 전처리

In [3]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [30]:
import jsonlines
dir = '../playground/data/coco2014_val_qa_eval/qa90_questions.jsonl'
img_dir = '../playground/data/eval/pope/val2014/COCO_val2014_'
# Open the JSONL file in read mode
with jsonlines.open(dir, 'r') as reader:
    # Iterate over each line in the file
    for line in reader:
        # Process each line as a JSON object    
        img_path = img_dir+line['image']
        base64_image = encode_image(img_path)
        line['answer'] = get_eval(line['text'], base64_image, 300)
        line['model'] = 'gpt-4-vision-preview'
        print(line)    
        break



{'question_id': 0, 'image': '000000441147.jpg', 'text': 'What is the color of the two suitcases in the image?', 'category': 'conv', 'answer': 'The suitcase on top has a tweed-like pattern with shades of brown and tan, and the one on the bottom is black with a textured surface. Both have brown handles and metal hardware.'}


In [5]:
import json

dir = '../playground/data/coco2014_val_qa_eval/qa90_questions.jsonl'
img_dir = '../playground/data/eval/pope/val2014/COCO_val2014_'
model_name = 'gpt-4-vision-preview'  # Model name
benchmark_name = 'coco2014_val_qa_eval'  # Benchmark name
language = 'en'
output_dir = f'./generated_data/[{model_name}]_[{benchmark_name}]_[{language}].json'  # Output file path

output_data = []  # List to hold the output data

# Open the JSONL file in read mode
with jsonlines.open(dir, 'r') as reader:
    # Iterate over each line in the file
    for line in reader:
        # Process each line as a JSON object    
        img_path = img_dir+line['image']
        base64_image = encode_image(img_path)
        line['answer'] = get_eval(line['text']+' Please answer in one sentence.', base64_image, 1500)
        line['answer'] = ' '.join(line['answer'].split()[:30])
        line['model'] = model_name
        print(line)
        output_data.append(line)  # Add the line to the output data
        # break

# Write the output data to a JSON file
with open(output_dir, 'w') as outfile:
    json.dump(output_data, outfile, indent=4, ensure_ascii=False)

{'question_id': 0, 'image': '000000441147.jpg', 'text': 'What is the color of the two suitcases in the image?', 'category': 'conv', 'answer': 'The suitcase on the top is brown with a tan pattern, and the one on the bottom is black with texture.', 'model': 'gpt-4-vision-preview'}
{'question_id': 1, 'image': '000000441147.jpg', 'text': 'Analyze the image in a comprehensive and detailed manner.', 'category': 'detail', 'answer': 'The image displays a collection of vintage suitcases featuring a variety of textures and hardware, with luggage tags and a note attached, suggesting a historical or travel-related theme.', 'model': 'gpt-4-vision-preview'}
{'question_id': 2, 'image': '000000441147.jpg', 'text': 'What potential factors could make these suitcases valuable?', 'category': 'complex', 'answer': 'The suitcases could be valuable due to their vintage nature, possible historical significance, quality of craftsmanship, or association with a notable brand or individual.', 'model': 'gpt-4-visio

In [13]:
import json
img_dir = '../playground/data/eval/pope/val2014/COCO_val2014_'
model_name = 'gpt-4-vision-preview'  # Model name
benchmark_name = 'coco2014_val_qa_eval'  # Benchmark name
language = 'ko'
dir = f'/data/MLP/cschoi/LLaVA/generation_data/generated_data/benchmark_data/long/[gpt-4-vision-preview]_[coco2014_val_qa_eval]_[ko].json'
output_dir = f'/data/MLP/cschoi/LLaVA/generation_data/generated_data/benchmark_data/[gpt-4-vision-preview30]_[coco2014_val_qa_eval]_[ko].json'  # Output file path

output_data = []  # List to hold the output data

# Open the JSON file in read mode
with open(dir, 'r') as json_file:
    data = json.load(json_file)
    # Iterate over each item in the list
    for line in data:
        # Process each item as a JSON object 
        img_path = img_dir+line['image']
        base64_image = encode_image(img_path)
        # a = get_eval(line['text'], base64_image, 1500)
        line['answer'] = get_eval(line['text']+' 한 문장으로 대답해주세요.', base64_image, 1500)
        line['answer'] = ' '.join(line['answer'].split()[:30])
        line['model'] = model_name
        print(line)
        output_data.append(line)  # Add the line to the output data


# Write the output data to a JSON file
# with open(output_dir, 'w') as outfile:
#     json.dump(output_data, outfile, indent=4, ensure_ascii=False)

{'question_id': 79, 'image': '000000460149.jpg', 'text': 'Describe the following image.', 'category': 'detail', 'answer': "This image shows a classic street clock on a sidewalk, offering a charming urban detail. The clock is mounted on a post with decorative elements and appears to be a traditional two-faced design, indicating it shows the time on both sides. The clock face is analog, with numbers going from 1 to 12, suggesting it follows the 12-hour format, and the hands on the clock show a specific time.\n\nSurrounding the clock, there are hanging flower baskets adding a touch of nature and beauty to the urban landscape. The sidewalk leads into a street scene that includes pedestrian crossing signs and parked bicycles in the background. Trees and additional planters with blooming flowers line the street, enhancing the picturesque nature of the setting. There's partial view of a person walking further down the sidewalk. The sky is clear and blue, indicating a sunny day. The overall at

In [14]:
from pprint import pprint
pprint(data)
output_dir = f'/data/MLP/cschoi/LLaVA/generation_data/generated_data/benchmark_data/[gpt-4-vision-preview]_[coco2014_val_qa_eval]_[en]_79.json'  # Output file path
with open(output_dir, 'w') as outfile:
    json.dump(output_data, outfile, indent=4, ensure_ascii=False)

[{'answer': 'The suitcase on the top appears to have a tan or light brown '
            'color with a striped pattern, while the suitcase on the bottom is '
            'black.',
  'category': 'conv',
  'image': '000000441147.jpg',
  'model': 'gpt-4-vision-preview',
  'question_id': 0,
  'text': 'What is the color of the two suitcases in the image?'},
 {'answer': 'The image shows two vintage suitcases stacked on top of each '
            'other. The top suitcase appears to be made of a patterned fabric, '
            'possibly a tweed-like material, and features leather corners and '
            'a leather handle that is at the center top of the suitcase. The '
            'clasps and lock mechanism on the top suitcase have a metallic '
            'finish, possibly made of brass, and it has a slightly rusted '
            'appearance which suggests age and previous usage.\n'
            '\n'
            "There's a tag attached to the top suitcase that seems to offer "
            'som